In [1]:
import pandas as pd
import numpy as np
from plotly import express as px
from matplotlib import pyplot as plt
import dill
from glob import glob
import re
from tqdm import tqdm

In [2]:
def load_models():
    model_dict = dict()
    for model_path in glob('../Models/*'):
        with open(model_path, 'rb') as f:
            model = model_path.split('\\')[-1].split('_')[0]
            model_dict[model] = {"model" : dill.load(f)}
            model_dict[model]['required_columns'] = model_dict[model]['model'].feature_names_in_
    return model_dict

In [3]:
models = load_models()

In [4]:
data = pd.read_csv('../Data/cleaned_data.csv', low_memory = False)
date_cols = ["EOMDate", "EOMOpenDt_YrMo", "EOMCloseDt_YrMo", "EOMMaturityDt_YrMo", "EOMDefaultDt_YrMO"]
data[date_cols] = data[date_cols].apply(pd.to_datetime)

In [5]:
def create_snapshot(data, snapshot_min_year = "2008-12-31"):
    closed_defaulted_acc = data.loc[(data.EOMCloseDt_YrMo < snapshot_min_year) | \
                                    (data.EOMDefaultDt_YrMO < snapshot_min_year), 'AcctID'].unique().tolist()

    snapshot_data = data[(data.EOMMaturityDt_YrMo >= snapshot_min_year) & 
                           (data.EOMOpenDt_YrMo <= snapshot_min_year) & 
                           ~(data.AcctID.isin(closed_defaulted_acc)) & 
                            (data.EOMDate >= snapshot_min_year)]
    
    return snapshot_data

In [6]:
snapshot_data = create_snapshot(data, "2008-12-31")

In [7]:
def get_all_predictions(snapshot_data, models):
    predictions = models['PA']['model'].predict_proba(snapshot_data[models['PA']['required_columns']])
    A_preds = pd.DataFrame(predictions, columns = ['A-A', 'A-B', 'A-C', 'A-D', 'A-E'], index = snapshot_data.index)

    predictions = models['PB']['model'].predict_proba(snapshot_data[models['PB']['required_columns']])
    B_preds = pd.DataFrame(predictions, columns = ['B-A', 'B-B', 'B-C', 'B-D', 'B-E'], index = snapshot_data.index)
    
    predictions = models['PC']['model'].predict_proba(snapshot_data[models['PC']['required_columns']])
    C_preds = pd.DataFrame(predictions, columns = ['C-A', 'C-B', 'C-C', 'C-D', 'C-E'], index = snapshot_data.index)
    
    EAD_preds = pd.DataFrame(models['EAD']['model'].predict(snapshot_data[models['EAD']['required_columns']]),
                                              columns = ['EAD_pred'], index = snapshot_data.index)
    LGD_preds = pd.DataFrame(models['LGD']['model'].predict(snapshot_data[models['LGD']['required_columns']]),
                                              columns = ['recovery_rate_pred'], index = snapshot_data.index)
    
    return pd.concat([snapshot_data, A_preds, B_preds, C_preds, EAD_preds, LGD_preds], axis = 1)

In [8]:
forecast_df = get_all_predictions(snapshot_data, models)

C:\Users\ajayd\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\ajayd\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\ajayd\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\ajayd\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\ajayd\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserW

In [9]:
forecast_df = forecast_df.sort_values(by = ['AcctID', 'EOMDate']).reset_index(drop = True)
acct_record_min_date = forecast_df.groupby(['AcctID'])['EOMDate'].min().reset_index()
forecast_df = forecast_df.merge(acct_record_min_date.rename(columns = {'EOMDate' : 'EOMFirst_Record_Date'}))

In [10]:
forecast_df['PA'] = np.NaN
forecast_df['PB'] = np.NaN
forecast_df['PC'] = np.NaN
forecast_df['PD'] = np.NaN
forecast_df['PE'] = np.NaN

In [11]:
# 95% of the loan accounts in the snapshot starts in A class. For sake of simplying the calculations we will assume the same
forecast_df.loc[forecast_df.EOMDate == forecast_df.EOMFirst_Record_Date, 
        ['PA', "PB", 'PC', 'PD', 'PE']] = forecast_df.loc[forecast_df.EOMDate == forecast_df.EOMFirst_Record_Date, 
                                                    ['A-A', "A-B", 'A-C', 'A-D', 'A-E']].values

In [12]:
abc = forecast_df.copy(deep = True)

In [13]:
for ind, row in tqdm(forecast_df.iterrows(), total = forecast_df.shape[0]):
    if not np.isnan(row['PA']):
        continue
    
    forecast_df.loc[ind, 'PA'] = (forecast_df.loc[ind - 1, 'PA'] * forecast_df.loc[ind, 'A-A'] +
                            forecast_df.loc[ind - 1, 'PB'] * forecast_df.loc[ind, 'B-A'] + 
                            forecast_df.loc[ind - 1, 'PC'] * forecast_df.loc[ind, 'C-A']) / \
                        (forecast_df.loc[ind - 1, 'PA'] + forecast_df.loc[ind - 1, 'PB'] + forecast_df.loc[ind - 1, 'PC'])
        
    forecast_df.loc[ind, 'PB'] = (forecast_df.loc[ind - 1, 'PA'] * forecast_df.loc[ind, 'A-B'] +
                            forecast_df.loc[ind - 1, 'PB'] * forecast_df.loc[ind, 'B-B'] + 
                            forecast_df.loc[ind - 1, 'PC'] * forecast_df.loc[ind, 'C-B']) / \
                         (forecast_df.loc[ind - 1, 'PA'] + forecast_df.loc[ind - 1, 'PB'] + forecast_df.loc[ind - 1, 'PC'])
        
    forecast_df.loc[ind, 'PC'] = (forecast_df.loc[ind - 1, 'PA'] * forecast_df.loc[ind, 'A-C'] +
                            forecast_df.loc[ind - 1, 'PB'] * forecast_df.loc[ind, 'B-C'] + 
                            forecast_df.loc[ind - 1, 'PC'] * forecast_df.loc[ind, 'C-C']) / \
                        (forecast_df.loc[ind - 1, 'PA'] + forecast_df.loc[ind - 1, 'PB'] + forecast_df.loc[ind - 1, 'PC'])
        
    forecast_df.loc[ind, 'PD'] = (forecast_df.loc[ind - 1, 'PA'] * forecast_df.loc[ind, 'A-D'] +
                            forecast_df.loc[ind - 1, 'PB'] * forecast_df.loc[ind, 'B-D'] + 
                            forecast_df.loc[ind - 1, 'PC'] * forecast_df.loc[ind, 'C-D']) / \
                        (forecast_df.loc[ind - 1, 'PA'] + forecast_df.loc[ind - 1, 'PB'] + forecast_df.loc[ind - 1, 'PC'])
        
    forecast_df.loc[ind, 'PE'] = (forecast_df.loc[ind - 1, 'PA'] * forecast_df.loc[ind, 'A-E'] +
                            forecast_df.loc[ind - 1, 'PB'] * forecast_df.loc[ind, 'B-E'] +
                            forecast_df.loc[ind - 1, 'PC'] * forecast_df.loc[ind, 'C-E']) / \
                        (forecast_df.loc[ind - 1, 'PA'] + forecast_df.loc[ind - 1, 'PB'] + forecast_df.loc[ind - 1, 'PC'])

100%|███████████████████████████████████████████████████████████████████████████| 25790/25790 [00:59<00:00, 430.89it/s]


In [16]:
forecast_df['ECL_pred'] = forecast_df['PD'] * forecast_df['EAD_pred'] * forecast_df['OpenAmt_YrMo'] * (1 - forecast_df['recovery_rate_pred'])

In [19]:
lgd = pd.read_csv('../Data/LGD.csv', usecols = ['AcctID', 'Default_Date', 'Actual_CO', 
                                                'RecoveryAmt','ChargeOff_Event', 'default_Balance'])
lgd['Default_Date'] = pd.to_datetime(lgd['Default_Date'])
lgd['default_Balance'] = lgd.default_Balance.str.replace("[^0-9.]",'', regex = True).astype(float)
lgd['Actual_CO'] = lgd.Actual_CO.str.replace("[^0-9.]",'', regex = True).astype(float)
lgd['RecoveryAmt'] = lgd.RecoveryAmt.str.replace("[^0-9.]",'', regex = True).astype(float)

lgd['recovery_rate'] = (lgd.default_Balance - lgd[['default_Balance', 'Actual_CO']].min(axis = 1) \
                        + lgd['RecoveryAmt']) \
                                    / lgd.default_Balance
lgd['Actual_CO'] = lgd['Actual_CO'].fillna(0)

lgd['recovery_rate'] = np.where(np.logical_and(lgd['Actual_CO'] <= 0, lgd['ChargeOff_Event'] == 1), 0, lgd['recovery_rate'])
lgd['recovery_rate'] = np.where(np.logical_and(lgd['Actual_CO'] <= 0, lgd['ChargeOff_Event'] == 0), 1, lgd['recovery_rate'])
lgd['recovery_rate'] = lgd['recovery_rate'].clip(0,1)
lgd['Actual_Loss'] = lgd['Actual_CO'] - lgd['RecoveryAmt']
lgd = lgd[['AcctID', 'Default_Date', 'recovery_rate', 'Actual_Loss']].fillna(0)

In [20]:
forecast_df = forecast_df.merge(lgd, left_on = ['AcctID', 'EOMDate'], right_on = ['AcctID', 'Default_Date'], how = 'left')

In [23]:
forecast_df['Balance_Ratio'] = forecast_df['Unpaid_Balance_Amt'] / forecast_df['OpenAmt_YrMo']

In [24]:
forecast_df.to_csv('../Data/forecast.csv', index = False)